In [1]:
import torch
from torch import nn
import numpy as np
from lgg_model import *
from gensim.models import Word2Vec

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path)
lgg_model.eval()

vanilla_GRU(
  (Embedding): Embedding(910, 50)
  (GRU): GRU(50, 100, num_layers=2, batch_first=True, dropout=0.5)
  (Linear): Linear(in_features=100, out_features=910, bias=True)
)

In [3]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [8]:
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
for i in lst:
    data = np.append(data, wv.key_to_index[i])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    data = np.stack((data,))
    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    lst.append(new_word)
    data = np.append(data, idx)

print('\nGeneration finished.')



临江仙
柳外轻池上雨，雨声碎荷声。小楼西角断明。一重风雨不飘零。 
渔家傲
小阳闲宴。绛想前面春归，有旧意如何。便暖催洗严妆面。 
减字木兰花
应黄钟寒气苦。冰生玉楼春
酒美春浓花世。相逢佳节。正值柳绵飞似无。天若有情天亦老。此恨无穷。 
更待高秋天气好。当时枝上落残花，今日春长相聚。 
蝶恋花
烟霏霏。风急雁行吹字断。红日短。江天雪意云撩乱。 
渔家傲
一流碧。新亭四面山相向。寂寞起来褰绣。月明正在梨花上。 
露和啼染无红雨。碧草绿杨岐路。况是长亭暮。 
玉楼春
花底闻敲两桨。女伴来寻访。酒盏旋将红绿水冰无断。 
朱阑夜夜无风，花影频移动。半醉腾腾春睡重。绿鬟堆枕香云拥。深深密阴初。折得花枝犹在手。春不住。惊起鸳鸯，只是红绡挂。颜色清新香洒。堪长。牡丹怎得称王。 
枕畔屏山围碧浪。金壶倒。花开未老人年少。 
渔家傲
四月芳林何悄悄，百月伤花却是啼妆。 
别来已隔千山翠。望断危楼斜日坠。关心只为牡丹红，一片春愁独落始开春。泪落金尊倾，岸。来如春梦几多时，去似朝人愁暖少。犹似当年醉里声。 
微雨后，薄腻烟光。老去风情应不到。凭阑是闲临眺。欲扫残红犹似仙。莫羡危月西催银箭。 
蝶恋花
留春满。往
Generation finished.
